In [2]:
import numpy as np
from scipy.io import loadmat
import matplotlib.pyplot as plt 
from Bio import SeqIO
import pandas as pd

def read_pseudo():
    file = SeqIO.parse('concatenated.fasta', 'fasta')
    data = [(record.id, record.seq._data) for record in file]
    seq = pd.DataFrame(data=data, columns=['id', 'sequence'])
    return seq

def read_staph():
    file = SeqIO.parse('core_gene_alignment-narsa.aln', 'fasta')
    data = [(record.id, record.seq._data) for record in file]
    seq = pd.DataFrame(data=data, columns=['id', 'sequence'])
    return staph

def pseudo_resp():
    responses= pd.read_csv('responses-pseudo.csv')
    return responses

def staph_resp():
    responses= pd.read_csv('responses-staph.csv')
    return responses




In [ ]:
pseudo=pseudo_resp()
pseudo['carb'] = pseudo['carb'].fillna(0)
resp=pseudo['carb']

allSeqs = []
for seq_record in SeqIO.parse('concatenated.fasta',  'fasta'):
      allSeqs.append(seq_record.seq)

seqMat = np.array(allSeqs)
print(seqMat.shape)
seqMat=seqMat[0:122,:]
print(seqMat.shape)

In [31]:
test=np.array(resp, dtype='object')
test*= 1
#changes all NA values to 0 too
new_lst = [[i] for i in test]
new_lst

[[1],
 [0],
 [1],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [1],
 [0],
 [1],
 [0],
 [0],
 [1],
 [1],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [1],
 [0],
 [0],
 [1],
 [0],
 [1],
 [1],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [1],
 [0],
 [1],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [1],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [1],
 [0],
 [1],
 [0],
 [0],
 [0],
 [0],
 [0],
 [1],
 [1],
 [0],
 [1],
 [0],
 [0],
 [1],
 [0],
 [0],
 [1],
 [0],
 [1],
 [1],
 [1],
 [1],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [1],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0]]

In [4]:
#one-hot encoding 
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

def hot_ohe(seq_array):
    label_encoder = LabelEncoder()
    integer_encoded_seq = label_encoder.fit_transform(seq_array)
    onehot_encoder = OneHotEncoder(sparse=False)
    integer_encoded_seq = integer_encoded_seq.reshape(len(integer_encoded_seq), 1)
    onehot_encoded_seq = onehot_encoder.fit_transform(integer_encoded_seq)
    return onehot_encoded_seq

def hot_ie(seq_array):
    label_encoder = LabelEncoder()
    integer_encoded_seq = label_encoder.fit_transform(seq_array)
    onehot_encoder = OneHotEncoder(sparse=False)
    integer_encoded_seq = integer_encoded_seq.reshape(len(integer_encoded_seq), 1)
    onehot_encoded_seq = onehot_encoder.fit_transform(integer_encoded_seq)
    return integer_encoded_seq

def ordinal_encoder(my_array):
    integer_encoded = label_encoder.transform(my_array)
    float_encoded = integer_encoded.astype(float)
    float_encoded[float_encoded == 0] = 0.25 # A
    float_encoded[float_encoded == 1] = 0.50 # C
    float_encoded[float_encoded == 2] = 0.75 # G
    float_encoded[float_encoded == 3] = 1.00 # T
    float_encoded[float_encoded == 4] = 0.00 # anything else, z
    return float_encoded

In [5]:
#from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
label_encoder.fit(np.array(['A','C','G','T','-']))

LabelEncoder()

In [6]:
a=ordinal_encoder(seqMat[0])

print(a.shape)
i=1
while i<122:
    b=ordinal_encoder(seqMat[i])
    a=np.vstack((a, b))
    i+=1

(483333,)


In [249]:
#practice
#a=ordinal_encoder(seqMat[0])
#b=ordinal_encoder(seqMat[1])
#a=np.vstack((a,b))


In [238]:
#nearest neighbor imputation
#from sklearn.impute import KNNImputer
#nan='-'
#imputer = KNNImputer(n_neighbors=2, weights="uniform")
#imputer.fit_transform(seqMat)

In [7]:
#print([key for key in in_data]) 
# -- use this line to see the keys in the dictionary data structure 
#linear classifier using feature calculation
x = a
y = new_lst
errorVariable=0
i=0

while i<122:
    x_eval2=x[i:i+12,:]
    y_eval2=y[i:i+12]
    x_train2=np.delete(x,np.s_[i:i+12],0)
    y_train2=np.delete(y,np.s_[i:i+12],0)
    w_opt2 = np.linalg.inv(x_train2.transpose()@x_train2)@x_train2.transpose()@y_train2
    y_hat2 = np.sign(x_eval2@w_opt2)
    error_vec3 = [0 if i[0]==i[1] else 1 for i in np.hstack((y_hat2, y_eval2))]
    errorVariable = errorVariable+sum(error_vec3)/12
    i+=12
    
print('Total number of errors: ' + str(errorVariable))

MemoryError: Unable to allocate 1.70 TiB for an array with shape (483333, 483333) and data type float64

In [8]:
#Psuedomonas SVM for all datasets
print('Calculating accuracy of SVM model (ordinal_encoding)')
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
x = a
y = new_lst
x_train=x[1:100,:]
y_train=y[1:100]
x_test=np.delete(x,np.s_[100:122],0)
y_test=np.delete(y,np.s_[100:122],0)

clf = SVC(kernel='linear')
clf.fit(x_train,np.ravel(y_train,order='C'))
y_pred = clf.predict(x_test)
print('Prediction accuracy for the first 22 items as testing set: '+str(accuracy_score(y_test,y_pred)))

x_train=x[22:122,:]
y_train=y[22:122]
x_test=np.delete(x,np.s_[0:22],0)
y_test=np.delete(y,np.s_[0:22],0)

clf = SVC(kernel='linear')
clf.fit(x_train,np.ravel(y_train,order='C'))
y_pred = clf.predict(x_test)
print('Prediction accuracy for the second 22 items as testing set: '+str(accuracy_score(y_test,y_pred)))

i=0
totalSum=0
while i<120:
    x_test=x[i:i+12,:]
    y_test=y[i:i+12]
    x_train=np.delete(x,np.s_[i:i+12],0)
    y_train=np.delete(y,np.s_[i:i+12],0)
    clf = SVC(kernel='linear')
    clf.fit(x_train,np.ravel(y_train,order='C'))
    y_pred = clf.predict(x_test)
    accuracy=accuracy_score(y_test,y_pred)
    totalSum+=accuracy
    i+=10
    
print('Prediction after dividing the data into ten segments for Psuedo: ' + str(totalSum/12))

Calculating accuracy of SVM model (ordinal_encoding)
Prediction accuracy for the first 22 items as testing set: 0.99
Prediction accuracy for the second 22 items as testing set: 1.0
Prediction after dividing the data into ten segments for Psuedo: 0.7847222222222222


In [9]:
#accuracy if we take encoding to be the ASCII values for the letters 
def ordinal_encoder2(my_array):
    integer_encoded = label_encoder.transform(my_array)
    float_encoded = integer_encoded.astype(float)
    float_encoded[float_encoded == 0] = 65 # A
    float_encoded[float_encoded == 1] = 67 # C
    float_encoded[float_encoded == 2] = 71 # G
    float_encoded[float_encoded == 3] = 84 # T
    float_encoded[float_encoded == 4] = 0 # anything else, z
    return float_encoded 

In [10]:
#aaccuracy if the encoding is changed to ASCII values instead of values that multiples of each other

print('Calculating accuracy of SVM model (ordinal_encoding with ASCII values)')

a2=ordinal_encoder2(seqMat[0])
i=1
while i<122:
    b2=ordinal_encoder2(seqMat[i])
    a2=np.vstack((a2, b2))
    i+=1

x = a2
y = new_lst
    
i=0
totalSum=0
while i<120:
    x_test=x[i:i+12,:]
    y_test=y[i:i+12]
    x_train=np.delete(x,np.s_[i:i+12],0)
    y_train=np.delete(y,np.s_[i:i+12],0)
    clf = SVC(kernel='linear')
    clf.fit(x_train,np.ravel(y_train,order='C'))
    y_pred = clf.predict(x_test)
    totalSum+=accuracy_score(y_test,y_pred)
    i+=10
    
print('Prediction after dividing the data into ten segments for Psuedo: ' + str(totalSum/12))


Calculating accuracy of SVM model (ordinal_encoding with ASCII values)
Prediction after dividing the data into ten segments for Psuedo: 0.8124999999999999


In [11]:
#nearest neighbor imputation
from sklearn.impute import KNNImputer
a2[a2 == 0] = 'nan'
nan='nan'
imputer = KNNImputer(n_neighbors=2, weights="uniform")
#stores the imputed array
a3=imputer.fit_transform(a2)
print(a3)

[[67. 65. 84. ... 65. 65. 65.]
 [67. 65. 84. ... 65. 84. 67.]
 [67. 65. 84. ... 65. 84. 67.]
 ...
 [65. 65. 65. ... 65. 84. 67.]
 [65. 65. 65. ... 65. 84. 67.]
 [65. 65. 65. ... 65. 84. 67.]]


In [12]:
#accuracy if the encoding is changed to ASCII values instead of values that multiples of each other
#and values are imputed based on near neighbor techniques

print('Calculating accuracy of SVM model (ordinal_encoding with imputation to ASCII values)')

x = a3
y = new_lst
    
i=0
totalSum=0
while i<120:
    x_test=x[i:i+12,:]
    y_test=y[i:i+12]
    x_train=np.delete(x,np.s_[i:i+12],0)
    y_train=np.delete(y,np.s_[i:i+12],0)
    clf = SVC(kernel='linear')
    clf.fit(x_train,np.ravel(y_train,order='C'))
    y_pred = clf.predict(x_test)
    totalSum+=accuracy_score(y_test,y_pred)
    i+=10
    
print('Prediction after dividing the data into twelve segments for Psuedo: ' + str(totalSum/12))


Calculating accuracy of SVM model (ordinal_encoding with imputation to ASCII values)
Prediction after dividing the data into twelve segments for Psuedo: 0.7847222222222222


In [13]:
#nearest neighbor imputation using nn=2, for ordinal_encoding
a[a == 0] = 'nan'
nan='nan'
imputer = KNNImputer(n_neighbors=2, weights="uniform")
#stores the imputed array
a4=imputer.fit_transform(a)

In [14]:
print('Calculating accuracy of SVM model (ordinal_encoding with imputation)')

x = a4
y = new_lst
    
i=0
totalSum=0
while i<120:
    x_test=x[i:i+12,:]
    y_test=y[i:i+12]
    x_train=np.delete(x,np.s_[i:i+12],0)
    y_train=np.delete(y,np.s_[i:i+12],0)
    clf = SVC(kernel='linear')
    clf.fit(x_train,np.ravel(y_train,order='C'))
    y_pred = clf.predict(x_test)
    totalSum+=accuracy_score(y_test,y_pred)
    i+=10
    
print('Prediction after dividing the data into twelve segments for Psuedo: ' + str(totalSum/12))


Calculating accuracy of SVM model (ordinal_encoding with imputation)
Prediction after dividing the data into twelve segments for Psuedo: 0.7569444444444445


In [15]:
#accuracy if the encoding is changed to ASCII values instead of values that multiples of each other
#and values are imputed based on near neighbor techniques, where nn = 5

print('Calculating accuracy of SVM model (ordinal_encoding with KNN (n=5) imputation to ASCII values)')
imputer = KNNImputer(n_neighbors=5, weights="uniform")

x = imputer.fit_transform(a2)
y = new_lst
    
i=0
totalSum=0
while i<120:
    x_test=x[i:i+12,:]
    y_test=y[i:i+12]
    x_train=np.delete(x,np.s_[i:i+12],0)
    y_train=np.delete(y,np.s_[i:i+12],0)
    clf = SVC(kernel='linear')
    clf.fit(x_train,np.ravel(y_train,order='C'))
    y_pred = clf.predict(x_test)
    totalSum+=accuracy_score(y_test,y_pred)
    i+=10
    
print('Prediction after dividing the data into twelve segments for Psuedo: ' + str(totalSum/12))


Calculating accuracy of SVM model (ordinal_encoding with KNN (n=5) imputation to ASCII values)
Prediction after dividing the data into twelve segments for Psuedo: 0.7777777777777778


In [32]:
#test for toby
resp2=pseudo['toby'].fillna(0)
toby=np.array(resp2, dtype='object')
toby*= 1
#changes all NA values to 0 too
new_lst2 = [[j] for j in toby]

In [33]:
#SVM testing for Toby responses with ordinal encoding with ASCII values 

print('Predicting SVM accuracy when considering Toby responses, for ordinal encoding with ASCII values[including imputation]')
x = a3
y = new_lst2
i=0
totalSum=0
while i<120:
    x_test=x[i:i+12,:]
    y_test=y[i:i+12]
    x_train=np.delete(x,np.s_[i:i+12],0)
    y_train=np.delete(y,np.s_[i:i+12],0)
    clf = SVC(kernel='linear')
    clf.fit(x_train,np.ravel(y_train,order='C'))
    y_pred = clf.predict(x_test)
    accuracy=accuracy_score(y_test,y_pred)
    totalSum+=accuracy
    i+=10
    
print('Prediction after dividing the data into ten segments for Psuedo: ' + str(totalSum/12))

print('Predicting SVM accuracy when considering Toby responses, for ordinal encoding including imputation')

x = a4
y = new_lst2

i=0
totalSum=0
while i<120:
    x_test=x[i:i+12,:]
    y_test=y[i:i+12]
    x_train=np.delete(x,np.s_[i:i+12],0)
    y_train=np.delete(y,np.s_[i:i+12],0)
    clf = SVC(kernel='linear')
    clf.fit(x_train,np.ravel(y_train,order='C'))
    y_pred = clf.predict(x_test)
    accuracy=accuracy_score(y_test,y_pred)
    totalSum+=accuracy
    i+=10
    
print('Prediction after dividing the data into ten segments for Psuedo: ' + str(totalSum/12))

Predicting SVM accuracy when considering Toby responses, for ordinal encoding with ASCII values[including imputation]
Prediction after dividing the data into ten segments for Psuedo: 0.9097222222222223
Predicting SVM accuracy when considering Toby responses, for ordinal encoding including imputation]
Prediction after dividing the data into ten segments for Psuedo: 0.9236111111111112


In [34]:

print('Calculating accuracy of SVM model (ordinal_encoding with KNN (n=5) imputation to ASCII values) for responses to Toby')
imputer = KNNImputer(n_neighbors=5, weights="uniform")

x = imputer.fit_transform(a2)
y = new_lst2
    
i=0
totalSum=0
while i<120:
    x_test=x[i:i+12,:]
    y_test=y[i:i+12]
    x_train=np.delete(x,np.s_[i:i+12],0)
    y_train=np.delete(y,np.s_[i:i+12],0)
    clf = SVC(kernel='linear')
    clf.fit(x_train,np.ravel(y_train,order='C'))
    y_pred = clf.predict(x_test)
    totalSum+=accuracy_score(y_test,y_pred)
    i+=10
    
print('Prediction after dividing the data into twelve segments for Psuedo: ' + str(totalSum/12))


Calculating accuracy of SVM model (ordinal_encoding with KNN (n=5) imputation to ASCII values) for responses to Toby
Prediction after dividing the data into twelve segments for Psuedo: 0.9097222222222223


In [36]:
#Begin calculation for Staphylococcus
allSeqs = []
for seq_record in SeqIO.parse('core_gene_alignment-narsa.aln','fasta'):
      allSeqs.append(seq_record.seq)

seqMat = np.array(allSeqs)
print(seqMat.shape)
seqMat=seqMat[0:125,:]
print(seqMat.shape)

(125, 983088)
(125, 983088)


In [4]:
staph=staph_resp()
respo=staph['resp'].fillna(0)
respo=np.array(respo, dtype='object')
respo*= 1
#changes all NA values to 0 too
new_lst3 = [[j] for j in respo]
new_lst3

[[0],
 [0],
 [0],
 [0],
 [0],
 [1],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [1],
 [0],
 [1],
 [0],
 [0],
 [0],
 [1],
 [0],
 [0],
 [0],
 [0],
 [1],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [1],
 [1],
 [1],
 [0],
 [0],
 [0],
 [1],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [1],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [1],
 [0],
 [0],
 [1],
 [1],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [1],
 [1],
 [0],
 [1],
 [0],
 [0],
 [1],
 [0],
 [0],
 [1],
 [0],
 [0],
 [1],
 [0],
 [0],
 [0],
 [0],
 [1],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [1],
 [0]]

In [43]:
#a=ordinal_encoder(seqMat[0])

#print(a.shape)
#i=1
#while i<122:
#    b=ordinal_encoder(seqMat[i])
#    a=np.vstack((a, b))
#    i+=1
    
#a[a == 0] = 'nan'
#nan='nan'
#imputer = KNNImputer(n_neighbors=2, weights="uniform")
#stores the imputed array
#a2=imputer.fit_transform(a)
#a2

In [5]:
pwd

'/home/aryan'